In [26]:
import cv2
import numpy as np

class ImageAligner:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        self.aligned_image = None

    def align_image(self):
        # Преобразуем изображение в оттенки серого
        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

        # Находим контуры на изображении
        contours, _ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Находим самый большой контур, который, вероятно, является границей изображения
        largest_contour = max(contours, key=cv2.contourArea)

        # Вычисляем угол поворота изображения
        rect = cv2.minAreaRect(largest_contour)
        angle = rect[2]

        # Поворачиваем изображение на вычисленный угол
        rows, cols = self.image.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        self.aligned_image = cv2.warpAffine(self.image, rotation_matrix, (cols, rows))

    def show_aligned_image(self):
        if self.aligned_image is not None:
            cv2.imshow("Выровненное изображение", self.aligned_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print("Изображение не было выровнено.")

# Пример использования
aligner = ImageAligner("Project1.jpg")
aligner.align_image()
aligner.show_aligned_image()

In [52]:


import cv2
import numpy as np

class ImageAligner:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"Не удалось загрузить изображение из {image_path}")
        self.aligned_image = None

    def align_image(self):
        # Преобразуем изображение в оттенки серого
        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

        # Находим контуры на изображении
        contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Находим самый большой контур, который, вероятно, является границей изображения
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)

            # Вычисляем угол поворота изображения
            rect = cv2.minAreaRect(largest_contour)
            angle = rect[2]

            # Поворачиваем изображение на вычисленный угол
            rows, cols = self.image.shape[:2]
            rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            self.aligned_image = cv2.warpAffine(self.image, rotation_matrix, (cols, rows))

    def show_aligned_image(self):
        if self.aligned_image is not None:
            cv2.imshow("Выровненное изображение", self.aligned_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print("Изображение не было выровнено.")

# Пример использования
try:
    aligner = ImageAligner("Project2.jpg")
    aligner.align_image()
    aligner.show_aligned_image()
except ValueError as e:
    print(e)